In [1]:
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder

In [2]:
original = pd.read_csv('Data_Train.csv', sep = ';')
df = original.copy()

In [ ]:
df.dropna(subset=['educacion'], inplace=True)

df.isnull().sum()

In [ ]:
# Reemplaza los valores nulos en las columnas especificadas con "Sin Información"
df[['tipo_contacto', 'resultado_campanas_anteriores']] = df[['tipo_contacto', 'resultado_campanas_anteriores']].fillna("Sin Información")

df.isnull().sum()

Buscamos modelo predictivo para nuestra columna "Target"

In [ ]:
df_train = df.copy()
df_train.sample(3)

,ID,edad,trabajo,estado_civil,educacion,deuda,saldo,vivienda,prestamo,tipo_contacto,duracion,fecha_contacto,campaign,tiempo_transcurrido,contactos_anteriores,resultado_campanas_anteriores,target
3380,3381,27,unemployed,soltero,secundaria/superiores,no,268,si,no,movil,188,5-may-2021,1,-1,0,Sin Información,no
26361,26362,35,blue-collar,divorciado,secundaria/superiores,no,3102,si,no,movil,138,20-nov-2021,1,-1,0,Sin Información,no
4983,4984,52,blue-collar,casado,priamaria,no,17,si,no,Sin Información,30,29-may-2021,2,-1,0,Sin Información,no


In [ ]:
# Supongamos que tienes un DataFrame llamado df y quieres contar los valores en la columna 'columna_deseada'
conteo_valores = df['educacion'].value_counts()
print(conteo_valores)

educacion
secundaria/superiores    22713
universitarios           13025
priamaria                 6708
Name: count, dtype: int64


In [ ]:
# Reemplaza el valor erróneo en la columna 'educacion'
df['educacion'] = df['educacion'].replace('priamaria', 'primaria')


In [ ]:
# Supongamos que tienes un DataFrame llamado df y quieres contar los valores en la columna 'columna_deseada'
conteo_valores = df['educacion'].value_counts()
print(conteo_valores)

educacion
secundaria/superiores    22713
universitarios           13025
primaria                  6708
Name: count, dtype: int64


In [ ]:
columns_to_delete = ['fecha_contacto', 'ID']
df_train.drop(columns=columns_to_delete,inplace=True)

df_train.sample(3)

,edad,trabajo,estado_civil,educacion,deuda,saldo,vivienda,prestamo,tipo_contacto,duracion,campaign,tiempo_transcurrido,contactos_anteriores,resultado_campanas_anteriores,target
20259,36,admin.,soltero,secundaria/superiores,no,16,no,no,Sin Información,119,2,-1,0,Sin Información,no
38465,48,management,soltero,universitarios,no,430,si,no,movil,44,1,-1,0,Sin Información,no
13296,36,services,casado,secundaria/superiores,no,647,si,si,movil,290,2,-1,0,Sin Información,no


In [ ]:

#Instancio el OneHot
onehot=OneHotEncoder()

# Aplico el OneHot a la columna job y guardo el resultao en a
onehot.fit(df_train[['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','tipo_contacto','resultado_campanas_anteriores']])
a=onehot.transform(df_train[['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','tipo_contacto','resultado_campanas_anteriores']])

# Convierto el a en data frame y lo llamo encoded_df
encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','tipo_contacto','resultado_campanas_anteriores']))

# Reseteo el índice de los dos data frames antes de concatenarlos
df_train.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# concateno los dos data frames y los guardo machacando el df
df_train=pd.concat([df_train,encoded_df],axis=1)

# Elimino la columna Job original antigua, la categórica
df_train.drop(columns=['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','tipo_contacto','resultado_campanas_anteriores'],inplace=True)

# Muestro tamaño yu sample del data frame df transformado
print(df_train.shape)
df_train.sample(5)

(42446, 38)


,edad,saldo,duracion,campaign,tiempo_transcurrido,contactos_anteriores,target,trabajo_admin.,trabajo_blue-collar,trabajo_entrepreneur,...,vivienda_si,prestamo_no,prestamo_si,tipo_contacto_Sin Información,tipo_contacto_fijo,tipo_contacto_movil,resultado_campanas_anteriores_Sin Información,resultado_campanas_anteriores_exito,resultado_campanas_anteriores_otro,resultado_campanas_anteriores_sin_exito
23854,36,1548,226,2,-1,0,no,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
35382,34,673,271,1,184,1,no,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
17564,30,48,144,2,187,1,no,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
27672,42,2816,193,2,-1,0,si,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
30687,41,431,196,1,88,5,si,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
X = df_train.drop(columns=["target"],inplace=False)
y = df_train["target"]

print("Tamaño X",X.shape)
print("Tamaño y",y.shape)

Tamaño X (42446, 37)
Tamaño y (42446,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
print("Tamaño X_train",X_train.shape)
print("Tamaño y_train",y_train.shape)
print("Tamaño X_test",X_test.shape)
print("Tamaño y_test",y_test.shape)

Tamaño X_train (31834, 37)
Tamaño y_train (31834,)
Tamaño X_test (10612, 37)
Tamaño y_test (10612,)


In [ ]:
# Suponiendo que X es tu conjunto de características y y es tu variable objetivo
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo de árbol de decisión:", accuracy)

Precisión del modelo de árbol de decisión: 0.8526196758386732


In [ ]:
# Estandarización controlada

estandarizador = StandardScaler()
estandarizador.fit(X_train)
X_train_std=estandarizador.transform(X_train)
X_test_std=estandarizador.transform(X_test)

In [ ]:
# 'model' es tu modelo entrenado DecisionTreeClassifier y 'df_train' es tu DataFrame original con las características de entrenamiento
# Selecciona todas las características (X) de tu DataFrame original 'df_train'
X_base_datos = df_train.drop('target', axis=1, inplace=False)  # Asegúrate de excluir la columna de la variable objetivo si está presente

# Realiza la predicción utilizando el modelo entrenado
y_pred_base_datos = model.predict(X_base_datos)

# Ahora 'y_pred_base_datos' contiene las predicciones para toda tu base de datos


In [ ]:
# utilizaremos la predicion de todo el df y lo agregamos en una nueva columna para comprar valores reales a los predichos

df['target_predicho'] = y_pred_base_datos

df.sample(3)

,ID,edad,trabajo,estado_civil,educacion,deuda,saldo,vivienda,prestamo,tipo_contacto,duracion,fecha_contacto,campaign,tiempo_transcurrido,contactos_anteriores,resultado_campanas_anteriores,target,target_predicho
3659,3660,44,entrepreneur,casado,universitarios,no,121,no,no,movil,248,9-nov-2021,1,91,1,exito,si,si
17691,17692,35,admin.,casado,secundaria/superiores,no,-203,si,no,movil,1649,11-jul-2021,2,-1,0,Sin Información,si,si
22220,22221,32,management,casado,universitarios,no,1402,no,no,Sin Información,52,1-jun-2021,1,557,2,sin_exito,no,no


In [ ]:
# Suponiendo que 'df' es tu DataFrame original con las columnas 'target' y 'predicciones'

# Contar los valores en la columna 'target'
conteo_target = df['target'].value_counts()

# Contar los valores en la columna 'predicciones'
conteo_predicciones = df['target_predicho'].value_counts()

# Imprimir los conteos
print("Conteo de valores en la columna 'target':")
print(conteo_target)
print("\nConteo de valores en la columna 'predicciones':")
print(conteo_predicciones)


Conteo de valores en la columna 'target':
target
no    37505
si     4941
Name: count, dtype: int64

Conteo de valores en la columna 'predicciones':
target_predicho
no    37389
si     5057
Name: count, dtype: int64


# Vamos a clasificar todas aquellas personas que tienen "si" en nuestra columna 'target' asi conoceremos mejor nuestros potenciales cliente

In [ ]:
# Buscaremos usar un metodo jerarquico para poder agrupar las personas que tengan "si" en nuestra columna 'target'

df_cluster = df.drop(columns=['ID', 'tipo_contacto', 'duracion', 'fecha_contacto', 'campaign', 'tiempo_transcurrido', 'contactos_anteriores', 'resultado_campanas_anteriores'], inplace=False)

df_cluster.sample(3)


,edad,trabajo,estado_civil,educacion,deuda,saldo,vivienda,prestamo,target,target_predicho
43604,30,blue-collar,casado,secundaria/superiores,no,360,si,si,no,no
31279,41,blue-collar,casado,secundaria/superiores,no,449,si,si,no,no
25201,42,blue-collar,casado,primaria,no,-407,si,si,si,si


In [ ]:
#Instancio el OneHot
onehot=OneHotEncoder()

# Aplico el OneHot a la columna job y guardo el resultao en a
onehot.fit(df_cluster[['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','target','target_predicho']])
a=onehot.transform(df_cluster[['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','target','target_predicho']])

# Convierto el a en data frame y lo llamo encoded_df
encoded_df = pd.DataFrame(a.toarray(), columns=onehot.get_feature_names_out(['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','target','target_predicho']))

# Reseteo el índice de los dos data frames antes de concatenarlos
df_cluster.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# concateno los dos data frames y los guardo machacando el df
df_cluster_train=pd.concat([df_cluster,encoded_df],axis=1)

# Elimino la columna original antigua, la categórica
df_cluster_train.drop(columns=['trabajo','estado_civil','educacion','deuda','vivienda','prestamo','target','target_predicho'],inplace=True)

# Muestro tamaño yu sample del data frame df transformado
print(df_cluster_train.shape)
df_cluster_train.sample(5)

(42446, 30)


,edad,saldo,trabajo_admin.,trabajo_blue-collar,trabajo_entrepreneur,trabajo_housemaid,trabajo_management,trabajo_retired,trabajo_self-employed,trabajo_services,...,deuda_no,deuda_si,vivienda_no,vivienda_si,prestamo_no,prestamo_si,target_no,target_si,target_predicho_no,target_predicho_si
22943,37,780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
7380,49,1381,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
39411,39,130,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
14307,47,1425,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
35075,38,359,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
# Guardaremos nuestro modelo para poder aplicarlo en otra base de datos llamada Data_test


# Entrena tu modelo con los datos de entrenamiento
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Guarda el modelo entrenado en un archivo
joblib.dump(model, 'modelo_entrenado.pkl')

# Luego, en otro script o en otro momento cuando necesites usar el modelo entrenado:

# Carga el modelo entrenado desde el archivo
# model_cargado = joblib.load('modelo_entrenado.pkl')

# Ahora puedes usar el modelo cargado para hacer predicciones en tus nuevas bases de datos
# y_pred_base1 = model_cargado.predict(X_base1)
# y_pred_base2 = model_cargado.predict(X_base2)

['modelo_entrenado.pkl']